In [1]:
from utils.df_handle import *

In [2]:
capture_sql = \
"""
SELECT terms,no_toi_han,updated_at,sum(tiennocongty)  as tiennocongty from (
	SELECT a.branchid,a.ordernbr,a.slsperid,a.slspername,c.tenquanlytt,c.tenquanlyvung,c.tenquanlykhuvuc,a.supname,a.asmname,a.dateoforder as ngaydatdon,
	Case when deli_last_updated ='1900-01-01 00:00:00' then now() -interval '1day' else deli_last_updated end
	as ngaygiaohang, a.custid,a.custname,b.tinh,a.terms,a.inchargename,a.tienchotso,a.tiengiaothanhcong,a.tiennocongty,a.tienthuquyxacnhan,a.duedate,
	Case when trangthaigiaohang ='NaN' then 'Chưa Xác Nhận' else a.trangthaigiaohang end as trangthaigiaohang,
	Case when trim(f.tencvbh) = trim(c.tenquanlykhuvuc) then f.email else 'nhanvt92@gmail.com'end as email_mng,
	Case when trim(e.tencvbh) = trim(c.tenquanlytt) then e.email else 'bimerap.main@gmail.com' end as email_sup,
	Case when trim(d.tencvbh) = trim(a.slspername) then d.email else 'bimerap.main@gmail.com' end as email_staff,
	
	Case 
			when a.terms = 'Gối 1 Đơn Hàng (trong 30 ngày)' then date(duedate)
			when trangthaigiaohang ='Đã giao hàng'  then date(a.deli_last_updated)  + interval '1 day' 		
	    when trangthaigiaohang not in ('Đã giao hàng') and a.terms in ('Thu tiền ngay không có VP PN','Gối Đầu 30 Pha Nam' )
	then date(dateoforder) + interval'2 day' 
			when trangthaigiaohang not in ('Đã giao hàng') and a.terms in ('Thu tiền ngay có VP PN')
	then date(dateoforder) + interval '1 day' 

	else now() 
	end as ngaytoihan1,
	Case when 
	( a.terms = 'Gối 1 Đơn Hàng (trong 30 ngày)' and date(duedate) <= date (now()) )
	or
	( a.trangthaigiaohang = 'Đã giao hàng'  and date(a.deli_last_updated)  + interval '1 day' <= date(now()) and a.terms not in ( 'Gối 1 Đơn Hàng (trong 30 ngày)' ) )
	
	or ( trangthaigiaohang not in ('Đã giao hàng') and a.terms in ('Thu tiền ngay không có VP PN','Gối Đầu 30 Pha Nam')
	and date(dateoforder) + interval'2 day' <= date(now() ) ) 

	or (trangthaigiaohang not in ('Đã giao hàng') and a.terms in ('Thu tiền ngay có VP PN')
	and date(dateoforder) + interval '1 day' <= date(now()) )
	
	then 'Đã tới hạn' else 'Chưa tới hạn' end as no_toi_han,
	a.inserted_at  as updated_at

	from f_tracking_debt a
	LEFT JOIN d_kh b on a.custid = b.custid
	LEFT JOIN d_users c on c.manv =a.slsperid
	LEFT JOIN d_takeorder_phanquyenstaff d on trim(d.tencvbh) = trim(a.slspername)
	LEFT JOIN d_takeorder_phanquyenstaff e on trim(e.tencvbh) = trim(c.tenquanlytt)
	LEFT JOIN d_takeorder_phanquyenstaff f on trim(f.tencvbh) = trim(c.tenquanlykhuvuc)
	where 
	(terms in ('Thu tiền ngay có VP PN','Thu tiền ngay không có VP PN','Gối 1 Đơn Hàng (trong 30 ngày)','Gối Đầu 30 Pha Nam')  
	--and a.paymentsform ='TM'--
	and debtincharge ='MDS'  and tiennocongty > 1000 and a.slsperid <> 'GH001') 
	) a
	GROUP BY
terms,no_toi_han,updated_at
"""

In [3]:
df = get_ps_df(capture_sql)

In [4]:
df.updated_at = datetime.now()

In [5]:
df.columns

Index(['terms', 'no_toi_han', 'updated_at', 'tiennocongty'], dtype='object')

In [6]:
df.tiennocongty = df.tiennocongty*1.1

In [7]:
pk = ['terms','no_toi_han','updated_at']

In [8]:
execute_values_upsert(df, "f_daily_capture_notoihan",pk)

INSERT INTO f_daily_capture_notoihan(terms,no_toi_han,updated_at,tiennocongty) VALUES %s ON CONFLICT (terms, no_toi_han, updated_at) DO UPDATE SET tiennocongty = excluded.tiennocongty
Data inserted using execute_values() successfully..
